In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    pipeline,
    logging,
)

from streaming_utils import TextStreamer

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_chat_alpaca/best_model/', 
    load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_chat_alpaca/best_model/'
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True, 
    skip_special_tokens=True, 
    # truncate_before_pattern=['[/'],
    truncate=True
)

In [4]:
print(tokenizer.eos_token)

<|endoftext|>


In [5]:
logging.set_verbosity(logging.CRITICAL)

In [6]:
template = """<|endoftext|>[INST] {prompt} [/INST]"""
eos_string = tokenizer.eos_token
history = None

In [7]:
print(template)

<|endoftext|>[INST] {prompt} [/INST]


In [8]:
while True:
    question=input("Question: ")
    inputs = ''

    prompt = history + '\n' + template.format(prompt=question, inputs=inputs) if history is not None else template.format(prompt=question, inputs=inputs)

    # print(f"PROMPT: {prompt}")

    prompt_tokenized = tokenizer(prompt, return_tensors='pt').to('cuda')['input_ids']
    
    output_tokenized = model.generate(
        input_ids=prompt_tokenized, 
        max_length=len(prompt_tokenized[0])+400,
        temperature=0.7,
        top_k=40,
        top_p=0.1,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )
    answer = tokenizer.decode(token_ids=output_tokenized[0][len(prompt_tokenized[0]):]).strip()
    
    if eos_string in answer:
        answer = answer.split(eos_string)[0].strip()
    if '[/' in answer:
        answer = answer.split('[/')[0].strip()

    history = '\n'.join([prompt, answer, eos_string])
    # print(f"ANSWER: {answer}\n")
    # print(f"HISTORY: {history}\n")
    print('#' * 50)

Question:  Hi.


/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Hello


error: unterminated character set at position 0